In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !git clone https://github.com/edgi-govdata-archiving/EEW-EJScreen.git
    %mv /content/EEW-EJScreen/ejscreen.db /content
else:
    print('Not running on CoLab')

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect( "ejscreen.db" )

sql = 'select Name, FIPS_Code, State_Code from states'
df0 = pd.read_sql_query(sql, conn)

In [ ]:
select_columns = 'P_LDPNT_D2, P_DSLPM_D2, P_CANCR_D2, P_RESP_D2, P_PTRAF_D2, P_PWDIS_D2, '
select_columns += 'P_PNPL_D2, P_PRMP_D2, P_PTSDF_D2, P_OZONE_D2, P_PM25_D2'

columns = ['P_LDPNT_D2', 'P_DSLPM_D2', 'P_CANCR_D2', 'P_RESP_D2', 'P_PTRAF_D2', 'P_PWDIS_D2', 
           'P_PNPL_D2', 'P_PRMP_D2', 'P_PTSDF_D2', 'P_OZONE_D2', 'P_PM25_D2']

all_columns = columns.copy()
all_columns.insert(0, 'State')
all_columns.insert(1, 'CD')

df_all = pd.DataFrame(columns=columns)

for index, row in df0.iterrows():
    state_fips = row['FIPS_Code'].zfill(2)
    state = row['State_Code']
    
    sql = 'select substr(id,6,2) as cd, {} from ejscreen where substr(id,1,2) = \'{}\' '
    sql = sql.format(select_columns, state_fips)
    df = pd.read_sql_query(sql, conn)
    # breakpoint()
    df[columns] = df[columns].apply(pd.to_numeric)
    df1 = df.groupby('cd')
    state_dict = {}
    for name, group in df1:
        # print( '{} - {}'.format(name, group))
        # df2 is dataframe of stdevs for each ejscreen index - P_LDPNT_D2, etc.
        df2 = df1.get_group(name).std()
        df2['State'] = state
        df2['CD'] = name
        df_all = df_all.append(df2.T,ignore_index=True)


In [ ]:
df_all['Lead_Paint.Pct'] = df_all['P_LDPNT_D2'].rank(pct=True)
df_all['Diesel.Pct'] = df_all['P_DSLPM_D2'].rank(pct=True)
df_all['Air_toxics_cancer.Pct'] = df_all['P_CANCR_D2'].rank(pct=True)
df_all['Air_toxics_resp.Pct'] = df_all['P_RESP_D2'].rank(pct=True)
df_all['Traffic.Pct'] = df_all['P_PTRAF_D2'].rank(pct=True)
df_all['Water_discharge.Pct'] = df_all['P_PWDIS_D2'].rank(pct=True)
df_all['NPL_sites.Pct'] = df_all['P_PNPL_D2'].rank(pct=True)
df_all['RMP_facilities.Pct'] = df_all['P_PRMP_D2'].rank(pct=True)
df_all['TSDF_facilities.Pct'] = df_all['P_PTSDF_D2'].rank(pct=True)
df_all['Ozone.Pct'] = df_all['P_OZONE_D2'].rank(pct=True)
df_all['PM2.5.Pct'] = df_all['P_PM25_D2'].rank(pct=True)

In [ ]:
df_all.to_csv('ej_screen_cd_stdev_rankings.csv')

In [ ]:
df_all

In [ ]:
df_all['State']